In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
tf.config.list_physical_devices()

In [ ]:
import numpy as np

X = np.array([-1,0,1,2,3,4])
y = np.array([-3,-1,1,3,5,7]) 

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

n1 = Dense(units=1, input_shape=[1]) # aby zobaczyc co dzieje sie w srodku
model = Sequential([n1])
model.compile(optimizer='sgd', loss="mean_squared_error")
model.fit(X,y, epochs=500)
print(n1.get_weights())

In [ ]:
print(model.predict(X),y)

In [ ]:
from tensorflow.keras.datasets import mnist
(tr_im, tr_lab),(te_im, te_lab) = mnist.load_data()

In [ ]:
network = Sequential()
network.add(Dense(512,activation='relu',input_shape=(28*28,)))
network.add(Dense(10, activation="softmax"))

In [ ]:
network.summary()

In [ ]:
digit = tr_im[5]
import matplotlib.pyplot as plt
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

In [ ]:
network.compile(optimizer='rmsprop', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])

tr_im = tr_im.reshape((60000, 28 * 28)) 
tr_im = tr_im.astype('float32') / 255
te_im = te_im.reshape((10000, 28 * 28))
te_im = te_im.astype('float32') / 255

In [ ]:
from tensorflow.keras.utils import to_categorical
tr_lab = to_categorical(tr_lab)
te_lab = to_categorical(te_lab)

In [ ]:
network.fit(tr_im, tr_lab, epochs=5, batch_size=128)

In [ ]:
te_loss, te_acc = network.evaluate(te_im, te_lab)
print(f'test_acc: {te_acc}')

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/Users/air/miniforge3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/air/miniforge3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c106detail19maybe_wrap_dim_slowExxb)
  Referenced from: '/Users/air/miniforge3/lib/python3.10/site-packages/torchvision/image.so'
  Expected in: '/Users/air/miniforge3/lib/python3.10/site-packages/torch/lib/libc10.dylib'
  warn(f"Failed to load image Python extension: {e}")


In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303160  [    0/60000]
loss: 2.290840  [ 6400/60000]
loss: 2.269083  [12800/60000]
loss: 2.263132  [19200/60000]
loss: 2.238932  [25600/60000]
loss: 2.213835  [32000/60000]
loss: 2.227447  [38400/60000]
loss: 2.191864  [44800/60000]
loss: 2.191319  [51200/60000]
loss: 2.155149  [57600/60000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 2.147101 

Epoch 2
-------------------------------
loss: 2.162020  [    0/60000]
loss: 2.152661  [ 6400/60000]
loss: 2.088309  [12800/60000]
loss: 2.104264  [19200/60000]
loss: 2.054948  [25600/60000]
loss: 2.001582  [32000/60000]
loss: 2.031145  [38400/60000]
loss: 1.949727  [44800/60000]
loss: 1.958149  [51200/60000]
loss: 1.894704  [57600/60000]
Test Error: 
 Accuracy: 54.7%, Avg loss: 1.878914 

Epoch 3
-------------------------------
loss: 1.914874  [    0/60000]
loss: 1.885920  [ 6400/60000]
loss: 1.754676  [12800/60000]
loss: 1.801479  [19200/60000]
loss: 1.708315  [25600/60000]
loss: 1.654907  [32000/600

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [14]:
import numpy as np

theta = np.pi/2
print(theta)
u = np.array([np.cos(theta),np.sin(theta)])
print(u)
v = np.array([-np.sin(theta), np.cos(theta)])
print(v)
R = np.array([u,v])
R

1.5707963267948966
[6.123234e-17 1.000000e+00]
[-1.000000e+00  6.123234e-17]


array([[ 6.123234e-17,  1.000000e+00],
       [-1.000000e+00,  6.123234e-17]])

In [15]:
R

array([[ 6.123234e-17,  1.000000e+00],
       [-1.000000e+00,  6.123234e-17]])

In [16]:
vec = np.array([2,1])

In [17]:
np.dot(R,vec)

array([ 1., -2.])

In [18]:
np.sin(theta)

1.0